In [54]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import random

In [55]:
df = pd.read_csv("covid_analytics_clinical_data.csv")
c_df = df[['Country','Study Pop Size (N)','Severity',]]
c_df = c_df.rename(columns={'Study Pop Size (N)': 'population'})
r_df = c_df.query('Country == "USA" & population >= 50')
r_df

,Country,population,Severity
258,USA,167.0,All
259,USA,101.0,All
260,USA,50.0,All
320,USA,53.0,All
427,USA,4103.0,All
428,USA,2104.0,Mild only
429,USA,1999.0,All
430,USA,1999.0,All
431,USA,650.0,Severe/Critical Only
473,USA,147.0,All
